In [1]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from torchinfo import summary
import pandas as pd
import os
from pathlib import Path

import NeuronalNetwork as CNNet

c:\Users\s8gre\AppData\Local\Programs\Python\Python39\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


Using cuda device


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(51136, 50)
        self.fc2 = nn.Linear(50, 2)


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #x = x.view(x.size(0), -1)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1)  


Using cuda device


In [3]:
def load_audio_file(path: str):

    dataset = []
    walker = sorted(str(p) for p in Path(path).glob(f'*.wav'))

    for i, file_path in enumerate(walker):
        path, filename = os.path.split(file_path)    
        # Load audio
        waveform, sample_rate = torchaudio.load(file_path)
        dataset.append([waveform, sample_rate])
        
    return dataset

In [4]:
model = CNNet()
model.load_state_dict(torch.load('./data/model.pth'))
model.eval()

trainset_speechcommand = load_audio_file('./testaudio')

testloader = torch.utils.data.DataLoader(trainset_speechcommand, batch_size=1,
                                            shuffle=True, num_workers=0)


In [5]:
spectrogram_tensor = None
directory = f'./data/testimage/order'
if(os.path.isdir(directory)):
    print("Data exists")
else:
    os.makedirs(directory, mode=0o777, exist_ok=True)
spectorgram_value = None
for data in testloader: 
    waveform = data[0]
    spectrogram_tensor = torchaudio.transforms.Spectrogram()(waveform)
    fig = plt.figure()
    fig = plt.figure()
    plt.imsave(f'./data/testimage/order/test.png', spectrogram_tensor[0].log2()[0,:,:].numpy(), cmap='viridis')
    

Data exists
[[-20.323214  -24.938257  -22.680481  ...  -9.652863  -12.82978
   -7.7300043]
 [-21.841885  -21.756824  -20.087286  ...  -8.05906   -10.274339
   -8.517631 ]
 [-21.279036  -23.433155  -18.039644  ...  -7.9789057  -8.450219
  -11.861316 ]
 ...
 [-26.869009  -25.95877   -25.062504  ... -24.650267  -28.011158
  -30.295326 ]
 [-40.719624  -24.163355  -25.175873  ... -25.848898  -27.623299
  -26.371065 ]
 [-25.931694  -23.238054  -26.589584  ... -23.138702  -25.016146
  -24.734066 ]]


TypeError: wrapped() missing 1 required positional argument: 'X'

<Figure size 640x480 with 0 Axes>

In [ ]:
dataset = datasets.ImageFolder(
    root="./data/testimage/",
    transform=transforms.Compose([transforms.Resize((201,81)),
                                  transforms.ToTensor()])
)
print(dataset[0])

In [ ]:
test_dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)


In [ ]:
classes = [
    "no",
    "yes"
]


In [ ]:
model.eval()
for batch, (X,Y) in enumerate(test_dataloader):
    pred = model(X)
    predicted = classes[pred[0].argmax(0)]
    print(f'Predicted: "{predicted}"')

Predicted: "no"
